In [58]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, coalesce
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType
from pyspark.sql.window import Window
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Pyspark playground") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()


In [59]:
df = spark.read.json("data/items.jl")

In [60]:
# drop unused columns
drop_cols = 'address direction distance renovate tel code'.split()
df = df.drop(*drop_cols)

In [61]:
# order columns
df = df.select('id name lat lng products province district subdistrict postcode'.split())

In [62]:
# drop ID duplicates (from scraping, since grid boundary might overlapped a bit)
df = df.dropDuplicates(['id'])

In [63]:
# # some record regions are NULL, but there's only 7 records so we SKIP it for now
# df.where(col('province').isNull()).show()
df = df.where(col('province').isNotNull())

In [33]:
df.show()

+-----+--------------------+-----------+------------+--------------------+-------------+----------------+-----------+--------+
|   id|                name|        lat|         lng|            products|     province|        district|subdistrict|postcode|
+-----+--------------------+-----------+------------+--------------------+-------------+----------------+-----------+--------+
|00326|   ประชาสงเคราะห์ 24|  13.775405|  100.563952|[[VF, 13, /static...|กรุงเทพมหานคร|          ดินแดง|     ดินแดง|   10320|
|00332|         ช่องลมนนทรี|  13.700048|  100.546238|[[VF, 13, /static...|กรุงเทพมหานคร|         ยานนาวา|  ช่องนนทรี|   10120|
|02070|       ปตท.ประชาชื่น|  13.835676|  100.539942|[[GP, 4, /static/...|กรุงเทพมหานคร|         บางซื่อ|    บางซื่อ|   10800|
|02943|        บางกะปิคอนโด|  13.772295|  100.644563|[[FM, 15, /static...|กรุงเทพมหานคร|         บางกะปิ|   คลองจั่น|   10240|
|03788|พหลโยธิน 2 ( New ...|13.77518416|100.54431991|[[VF, 13, /static...|กรุงเทพมหานคร|           พญาไท|   สาม

# Crunch

## Groupby region

In [37]:
groupby_region = df.groupBy('province district subdistrict'.split()).agg(F.count('*').alias('count'))
groupby_region.show()

+-------------+-----------------+-------------+-----+
|     province|         district|  subdistrict|count|
+-------------+-----------------+-------------+-----+
|กรุงเทพมหานคร|           พระนคร|     บวรนิเวศ|    4|
|  สมุทรปราการ|           บางพลี|      บางโฉลง|   30|
|      นนทบุรี|        บางบัวทอง|         ลำโพ|    8|
|กรุงเทพมหานคร|        ภาษีเจริญ|      บางด้วน|    8|
|กรุงเทพมหานคร|        ลาดกระบัง|คลองสามประเวศ|    8|
|  สมุทรปราการ|           บางบ่อ|   บางพลีน้อย|    4|
|   ฉะเชิงเทรา|          บางปะกง|        พิมพา|    3|
|    สมุทรสาคร|   เมืองสมุทรสาคร|     โกรกกราก|    2|
|   ฉะเชิงเทรา|    บางน้ำเปรี้ยว|      ศาลาแดง|    1|
|กรุงเทพมหานคร|          สวนหลวง|      สวนหลวง|   85|
|กรุงเทพมหานคร|          คลองเตย|      คลองตัน|   16|
|    สมุทรสาคร|   เมืองสมุทรสาคร|      ท่าฉลอม|    3|
|กรุงเทพมหานคร|ป้อมปราบศัตรูพ่าย|     ป้อมปราบ|   12|
|     ปทุมธานี|          ธัญบุรี|  บึงน้ำรักษ์|    6|
|กรุงเทพมหานคร|        ลาดกระบัง|    ลาดกระบัง|   31|
|      ราชบุรี|         บ้าน

In [38]:
# groupby_region.toPandas().to_csv('data/output/groupby_region.csv', index=False)

## Groupby product type

In [65]:
df = df\
    .withColumn('products', F.explode('products'))\
    .select(*df.columns,
            col('products.code').alias('product_code'),
            col('products.name').alias('product_name')
           )\
    .drop('products')\
    .cache()

In [52]:
groupby_region_product = df.groupBy('province district subdistrict product_name'.split()).agg(F.count('*').alias('count'))
groupby_region_product.show()

+-------------+----------------+-----------+-----------------+-----+
|     province|        district|subdistrict|     product_name|count|
+-------------+----------------+-----------+-----------------+-----+
|กรุงเทพมหานคร|         คลองสาน| คลองต้นไทร|           คัดสรร|   11|
|กรุงเทพมหานคร|         ปทุมวัน|    วังใหม่|    ผักและผลไม้สด|   13|
|กรุงเทพมหานคร|           บางนา| บางนาเหนือ|   เอ็กซ์ต้า พลัส|    3|
|กรุงเทพมหานคร|         ทุ่งครุ|      บางมด|              กัฟ|    9|
|กรุงเทพมหานคร|        ตลิ่งชัน| คลองชักพระ|       ออลล์คาเฟ่|    2|
|  สมุทรปราการ|เมืองสมุทรปราการ|  บางปูใหม่|       ออลล์คาเฟ่|   12|
|       นครปฐม|     เมืองนครปฐม|โพรงมะเดื่อ|    ผักและผลไม้สด|    2|
|กรุงเทพมหานคร|          ดินแดง|     ดินแดง|       ออลล์คาเฟ่|   68|
|กรุงเทพมหานคร|          ธนบุรี|   ดาวคะนอง|เนื้อสัตว์แช่แข็ง|    3|
|       นครปฐม|       นครชัยศรี|  ท่าตำหนัก|       ออลล์คาเฟ่|    4|
|  สมุทรปราการ|          บางบ่อ|  บางเพรียง|เนื้อสัตว์แช่แข็ง|    2|
|      นนทบุรี|         ไทรน้อย|  

In [53]:
groupby_region_product.toPandas().to_csv('data/output/groupby_region_product.csv', index=False)

# Count product groups per branch

In [66]:
groupby_region_id_product = df.groupBy('id province district subdistrict lat lng'.split()).agg(F.count('product_name').alias('count'))
groupby_region_id_product.show()

+-----+-------------+----------------+-----------+-----------+------------+-----+
|   id|     province|        district|subdistrict|        lat|         lng|count|
+-----+-------------+----------------+-----------+-----------+------------+-----+
|00326|กรุงเทพมหานคร|          ดินแดง|     ดินแดง|  13.775405|  100.563952|    3|
|00332|กรุงเทพมหานคร|         ยานนาวา|  ช่องนนทรี|  13.700048|  100.546238|    4|
|02070|กรุงเทพมหานคร|         บางซื่อ|    บางซื่อ|  13.835676|  100.539942|    1|
|02943|กรุงเทพมหานคร|         บางกะปิ|   คลองจั่น|  13.772295|  100.644563|    5|
|03788|กรุงเทพมหานคร|           พญาไท|   สามเสนใน|13.77518416|100.54431991|    4|
|04319|กรุงเทพมหานคร|          บางเขน| อนุสาวรีย์|13.85987759|100.62219441|    2|
|04868|กรุงเทพมหานคร|         จตุจักร|  จันทรเกษม|  13.821345|  100.581538|    4|
|04879|  สมุทรปราการ|เมืองสมุทรปราการ| สำโรงเหนือ|  13.651742|   100.61862|    3|
|05176|กรุงเทพมหานคร|      วังทองหลาง|    พลับพลา| 13.7623873|100.61675226|    1|
|05236|กรุงเทพมห

In [67]:
groupby_region_id_product.toPandas().to_csv('data/output/groupby_region_id_product.csv', index=False)